# **CNN 練習**

## 匯入所需套件

In [ ]:
# tf.keras.utils.plot_model 需要安裝的套件
!pip install pydot
!pip install graphviz

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Dense, Dropout,
                                     Activation, Flatten, Conv2D,
                                     MaxPooling2D)
from tensorflow.keras.utils import plot_model

## Cifar10 資料讀入及前處理

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train shape:', x_train.shape)
print('y_train.shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# x_train.shape: 四個維度：第 1 維度為筆數、第 2, 3 維度為影像大小 32*32、第 4 維度是 RGB 三原色，所以是 3
# x_train 中有 50000 筆訓練資料，以及 x_test 中有 10000 筆的測試資料

In [ ]:
# image preprocessing
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# 將 features(照片影像特徵值)標準化，可以提高模型預測的準確度，並且更快收斂
x_train /= 255  # rescaling
x_test /= 255  # rescaling

In [ ]:
# 將訓練資料與測試資料的label，進行Onehot encoding轉換
num_classes = 10
y_train = np.eye(num_classes, dtype='float32')[y_train[:, 0]]
y_test = np.eye(num_classes, dtype='float32')[y_test[:, 0]]

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

## 模型定義
- 試著建立圖中的模型架構

![image](https://hackmd.io/_uploads/By2IG1PL6.png)

In [ ]:
print(x_train.shape[1:])

In [ ]:
'''在__________填入正確的參數讓產生的卷積影像大小不變吧'''

inputs = Input(shape=x_train.shape[1:])
# 建立卷積層，設定 32 個 3*3 的filters
# 設定 padding，讓卷積運算，產生的卷積影像大小不變
# 所有激活函數都設定為 ReLU
x = Conv2D('______', '______', padding='______', activation='______')(inputs)
x = Dropout(rate=0.25)(x)

# 第二層 - 卷積層 (3x3 的 filters) + 池化層
x = Conv2D('______', '______', padding='______', activation='______')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# 第三層 - 卷積層 (3x3 的 filters)
x = Conv2D('______', '______', padding='______', activation='______')(x)

# 第四層 - 卷積層 (3x3 的 filters) + 池化層
x = Conv2D('______', '______', padding='______', activation='______')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)   # 避免overfitting

# 建立分類模型 (MLP) : 平坦層 + 隱藏層 (512 神經元, ReLU 為激活函數) + 輸出層 (10)
x = Flatten()(x)
x = Dense('______', activation='______')(x)
x = Dropout(0.25)(x)
outputs = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=inputs, outputs=outputs)

In [ ]:
# model.summary()
plot_model(model, to_file='ex_Model.png', show_shapes=True)

In [ ]:
model_dir = 'model-logs/'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

logfiles = f'{model_dir}{model.__class__.__name__}'

modelfiles = f'{model_dir}basic_model-best-model.h5'
model_mckp = keras.callbacks.ModelCheckpoint(modelfiles,
                                             monitor='val_accuracy',
                                             save_best_only=True)

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss',
                                          patience=5,
                                          verbose=1)


callbacks_list = [model_mckp, earlystop]

In [ ]:
# 編譯模型
# 選用 Adam 為 optimizer
learning_rate = 0.0001
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

## 開始訓練模型

In [ ]:
batch_size = 32
epochs = 20
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

## 測試資料

In [ ]:
best_model = "./model-logs/basic_model-best-model.h5"
model = load_model(best_model)

In [ ]:
test_pred = model.predict(x_test[0:1]).argmax(-1)

plt.imshow(x_test[0])
print('prediction: ', test_pred)

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=2)

In [ ]:
y_pred = model.predict(x_test)
print(y_pred[:3])
y_pred = y_pred.argmax(-1)
print(y_pred[:3])

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(accuracy_score(y_test.argmax(-1), y_pred))
print(confusion_matrix(y_test.argmax(-1), y_pred))

## 訓練結果視覺化

In [ ]:
train_history = ['loss', 'val_loss', 'accuracy', 'val_accuracy']
name_history = ['training_loss', 'val_loss', 'training_acc', 'val_acc']

plt.figure(figsize=(12, 5))
for eachx, eachy, i in zip(train_history, name_history, range(4)):
    if i % 2 == 0:
        plt.subplot(1, 2, i//2+1)
    l_x = len(history.history[eachx])
    plt.plot(np.arange(l_x), history.history[eachx], label=eachy)
    plt.legend(loc='best')
    plt.title(eachy)
plt.show()